In [1]:
##Importando librerias
import pandas as pd
import requests
from icalendar import Calendar
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
### Cargando datos desde google sheets

# Ruta local al archivo JSON que subiste
ruta_json = "/Users/ricardomosqueira/airbnb_dashboard/sheets-api-calendar-0046b74b266e.json"

# Scopes para Google Sheets y Google Drive
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Credenciales
creds = ServiceAccountCredentials.from_json_keyfile_name(ruta_json, scope)
client = gspread.authorize(creds)

# Abrir el Google Sheet por nombre
spreadsheet = client.open("PRUEBA DATA OPS -  Analysis")

# Seleccionar la hoja
sheet = spreadsheet.worksheet("PRUEBA DATA OPS")

# Cargar los datos como DataFrame
data = sheet.get_all_records()
rocketIA_data = pd.DataFrame(data)

# Mostrar las primeras filas
print(rocketIA_data.head())


        Fecha          Producto            Canal  Gasto_MXN  Clicks     CPA  \
0  07/05/2025   Tarjeta Crédito           TikTok     978.05     102  140.85   
1  04/05/2025     Cuenta Nómina  Email Marketing     676.87     123   67.58   
2  13/05/2025   Tarjeta Crédito  Email Marketing     840.23      95  100.58   
3  15/05/2025   Tarjeta Crédito           TikTok     219.27      62   71.43   
4  11/05/2025  Crédito Personal       Google Ads     875.81     214  153.81   

    ROI     CTR      Estatus        Motivo_Rechazo  Leads_Obtenidos  \
0  1.35  0.0727   Completado                   N/A                6   
1  1.76  0.0175   Completado                   N/A               11   
2  0.90  0.0877   Completado                   N/A                3   
3  2.15  0.0533  En progreso  Historial crediticio               15   
4  2.99  0.0162  En progreso                   N/A                1   

   Conversión_Funnel_%  
0                 0.21  
1                 0.06  
2                 0.09 

In [12]:
rocketIA_data.groupby(['Estatus','Motivo_Rechazo'])[['Leads_Obtenidos']].sum()  

Leads_Obtenidos
Estatus     Motivo_Rechazo                           
Completado  Documentación incompleta              492
            Historial crediticio                  271
            Ingreso insuficiente                  340
            N/A                                  4067
En progreso Documentación incompleta              349
            Historial crediticio                  231
            Ingreso insuficiente                  247
            N/A                                  1972
Rechazado   Documentación incompleta              141
            Historial crediticio                  222
            Ingreso insuficiente                  173
            N/A                                  1639

In [13]:
def clasificar_lead(row):
    if row['Estatus'] == "Completado" and row['Motivo_Rechazo'] == "N/A":
        return "Exitoso"
    elif row['Estatus'] == "En progreso" and row['Motivo_Rechazo'] == "N/A":
        return "En progreso"
    else:
        return "Rechazado"

rocketIA_data['lead_status'] = rocketIA_data.apply(clasificar_lead, axis=1)


In [14]:
rocketIA_data.groupby(['lead_status'])[['Leads_Obtenidos']].sum()

,Leads_Obtenidos
lead_status,
En progreso,1972
Exitoso,4067
Rechazado,4105


In [9]:
### CPA and ROI per channel and product 

rocketIA_data['Fecha'] = pd.to_datetime(rocketIA_data['Fecha'], format="%d/%m/%Y")
rocketIA_data.groupby(['Fecha','Canal','Producto'])[['CPA','ROI']].mean()

CPA       ROI
Fecha      Canal           Producto                              
2025-05-01 Email Marketing Crédito Personal  128.461111  1.605556
                           Cuenta Nómina     143.588750  1.836250
                           Tarjeta Crédito   121.227143  1.561429
           Facebook        Crédito Personal   81.464286  1.774286
                           Cuenta Nómina     130.193333  1.026667
...                                                 ...       ...
2025-05-15 Google Ads      Cuenta Nómina     123.044286  1.840000
                           Tarjeta Crédito   138.312500  1.480000
           TikTok          Crédito Personal  118.722000  1.870000
                           Cuenta Nómina     106.483333  1.073333
                           Tarjeta Crédito   100.255714  1.610000

[179 rows x 2 columns]